Implement linear regression by using high level api's of deep learning frameworks

## Generating dataset

In [1]:
from d2l import mxnet as d2l 
from mxnet import np, npx, autograd, gluon
npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
features[0], labels[0]

(array([1.1630785, 2.2122061]), array([-1.0015316]))

## Reading dataset

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[array([[ 1.6076493 ,  0.51737875],
        [ 0.5712682 ,  0.57461417],
        [ 0.5035904 , -0.97121936],
        [ 0.1984881 , -0.18156298],
        [ 1.4290679 , -0.04376583],
        [-0.36128756, -1.3773832 ],
        [-0.89261854, -0.2328267 ],
        [ 0.73395604,  2.739364  ],
        [ 0.08167123,  1.1786946 ],
        [-0.8413213 , -0.686307  ]]),
 array([[ 5.665324  ],
        [ 3.3854222 ],
        [ 8.518152  ],
        [ 5.2185535 ],
        [ 7.1972    ],
        [ 8.150142  ],
        [ 3.2000787 ],
        [-3.6464229 ],
        [ 0.35533163],
        [ 4.837474  ]])]

## Defining the model

Dense--defines fully connected layer 

our linear model has only 1 layer

In [4]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

## Initializing model parameters
Initializing weight from normal distribution with mean 0 and standard deviation 0.01. 

Bias will be automatically initialized to 0 

import initializer module from mxnet (abbrevated init)

behind the scene, initialization is actually deferred and the input dimensionality information will be provided when we pass the parameters first time. Since the parameters have not been initialized yet, we cannot access or manipulate them.

In [5]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

## Defining the loss function
In Gluon, the loss module defines various loss functions. 

In [9]:
loss = gluon.loss.L2Loss()


## Defining the Optimization algorithm
Minibatch stochastic gradient descent is a standard tool for optimizing neural networks. In Gluon, the Trainer class defines number of variations on this algorithm. 

In [7]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':0.03})

## Training

For each minibatch:

    1. Generate predictions by calling net(X) and calculate loss l (the forward propagation)
    2. Calculate gradients by running the backpropagation
    3. Update the model parameters by invoking our optimizer


In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(1)
    l=loss(net(features), labels).mean()
    print(f'epoch {epoch +1}, loss {l.asnumpy():f}')
    

epoch 1, loss 0.000052
epoch 2, loss 0.000052
epoch 3, loss 0.000052


In [11]:
w=net[0].weight.data()
print(f'error in estimating b: {true_w -w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b-b}')

error in estimating b: [ 0.00093758 -0.00140834]
error in estimating b: [0.00069189]


Help on Sequential in module mxnet.gluon.nn.basic_layers object:

class Sequential(mxnet.gluon.block.Block)
 |  Sequential(prefix=None, params=None)
 |  
 |  Stacks Blocks sequentially.
 |  
 |  Example::
 |  
 |      net = nn.Sequential()
 |      # use net's name_scope to give child Blocks appropriate names.
 |      with net.name_scope():
 |          net.add(nn.Dense(10, activation='relu'))
 |          net.add(nn.Dense(20))
 |  
 |  Method resolution order:
 |      Sequential
 |      mxnet.gluon.block.Block
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, prefix=None, params=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  add(self, *blocks)
 |      Adds block on top of the stack.
 |  
 |  forward(self, x)
 |      Overrides to implement forward computation using :py:class:`NDArray`. Only
 |      accepts positional 